In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000) ## 800GB?

from calitp.tables import tbl
from calitp import query_sql, magics
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd

import datetime as dt
import time
from zoneinfo import ZoneInfo

import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

from IPython.display import display, Markdown


/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. Yo

In [2]:
import utils

In [6]:
import altair as alt
from dla_utils import _dla_utils

from shared_utils import altair_utils
from shared_utils import geography_utils
from shared_utils import calitp_color_palette as cp
from shared_utils import styleguide

In [3]:
rt_sched = query_sql(
    """
SELECT *
FROM `cal-itp-data-infra-staging.natalie_views.gtfs_rt_vs_sched_r`
"""
)

/opt/conda/lib/python3.10/site-packages/google/auth/_default.py:79: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/


In [5]:
rt_sched['service_date'] = pd.to_datetime(rt_sched['service_date'])
rt_sched['weekday'] = pd.Series(rt_sched.service_date).dt.day_name()
rt_sched['month'] =  pd.Series(rt_sched.service_date).dt.month_name()

In [12]:
rt_sched.sample(5)

,calitp_itp_id,calitp_url_number,route_id,service_date,num_sched,num_rt,pct_ran,weekday,month
907,300,0,3493,2022-05-04,95,94,0.99,Wednesday,May
74,300,0,3481,2022-06-08,153,142,0.93,Wednesday,June
909,300,0,3485,2022-05-14,134,94,0.70,Saturday,May
829,300,0,3496,2022-06-07,84,84,1.00,Tuesday,June
362,300,0,3500,2022-05-09,35,30,0.86,Monday,May


In [40]:
(rt_sched
 >>group_by(_.calitp_itp_id,
            _.calitp_url_number,
            _.service_date,
            _.weekday,
           _.month
           )
 >>summarize(total_num_sched = _.num_sched.sum() ,
             total_num_rt = _.num_rt.sum())
 >>mutate(pct_ran = (_.total_num_rt)/(_.total_num_sched))
            )


,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_ran
0,300,0,2022-05-01,Sunday,May,771,527,0.68
1,300,0,2022-05-02,Monday,May,1519,1069,0.70
2,300,0,2022-05-03,Tuesday,May,1624,1473,0.91
3,300,0,2022-05-04,Wednesday,May,1622,1487,0.92
4,300,0,2022-05-05,Thursday,May,1628,1455,0.89
...,...,...,...,...,...,...,...,...
56,300,0,2022-06-26,Sunday,June,772,500,0.65
57,300,0,2022-06-27,Monday,June,1465,1017,0.69
58,300,0,2022-06-28,Tuesday,June,1559,1457,0.93
59,300,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [57]:
def by_date(df, sum1_sched, sum2_rt):
    agg_df = (df
     >>group_by(_.calitp_itp_id,
            _.calitp_url_number,
            _.service_date,
            _.weekday,
           _.month)
     >>summarize(total_num_sched = (_[sum1_sched].sum()),
             total_num_rt = (_[sum2_rt].sum()))
     >>mutate(pct_ran = (_.total_num_rt)/(_.total_num_sched))
            )
    return agg_df


In [58]:
# cols_to_agg = ['calitp_itp_id', 'calitp_url_number','service_date', 'weekday', 'month']

In [60]:
rt_sched_agg = by_date(rt_sched, 'num_sched', 'num_rt')

In [61]:
rt_sched_agg

,calitp_itp_id,calitp_url_number,service_date,weekday,month,total_num_sched,total_num_rt,pct_ran
0,300,0,2022-05-01,Sunday,May,771,527,0.68
1,300,0,2022-05-02,Monday,May,1519,1069,0.70
2,300,0,2022-05-03,Tuesday,May,1624,1473,0.91
3,300,0,2022-05-04,Wednesday,May,1622,1487,0.92
4,300,0,2022-05-05,Thursday,May,1628,1455,0.89
...,...,...,...,...,...,...,...,...
56,300,0,2022-06-26,Sunday,June,772,500,0.65
57,300,0,2022-06-27,Monday,June,1465,1017,0.69
58,300,0,2022-06-28,Tuesday,June,1559,1457,0.93
59,300,0,2022-06-29,Wednesday,June,1548,1454,0.94


In [77]:
rt_sched>>count(_.route_id)>>arrange(-_.n)

,route_id,n
0,3479,61
1,3480,61
2,3481,61
4,3485,61
5,3486,61
6,3487,61
8,3489,61
9,3490,61
12,3493,61
13,3494,61


In [76]:
(rt_sched>>filter(_.route_id=='3501')>>arrange(_.service_date))

,calitp_itp_id,calitp_url_number,route_id,service_date,num_sched,num_rt,pct_ran,weekday,month
186,300,0,3501,2022-05-02,104,0,0.00,Monday,May
197,300,0,3501,2022-05-03,104,0,0.00,Tuesday,May
192,300,0,3501,2022-05-04,104,0,0.00,Wednesday,May
205,300,0,3501,2022-05-05,104,0,0.00,Thursday,May
171,300,0,3501,2022-05-06,52,0,0.00,Friday,May
188,300,0,3501,2022-05-09,104,0,0.00,Monday,May
208,300,0,3501,2022-05-10,104,0,0.00,Tuesday,May
204,300,0,3501,2022-05-11,104,0,0.00,Wednesday,May
210,300,0,3501,2022-05-12,104,0,0.00,Thursday,May
180,300,0,3501,2022-05-13,52,0,0.00,Friday,May


In [84]:
(rt_sched>>filter(_.route_id=='3488')
).head()

,calitp_itp_id,calitp_url_number,route_id,service_date,num_sched,num_rt,pct_ran,weekday,month
0,300,0,3488,2022-05-13,8,8,1.00,Friday,May
1,300,0,3488,2022-06-16,8,8,1.00,Thursday,June
211,300,0,3488,2022-06-27,6,4,0.67,Monday,June
212,300,0,3488,2022-05-31,6,4,0.67,Tuesday,May
213,300,0,3488,2022-05-16,6,4,0.67,Monday,May


In [86]:
line = alt.Chart(rt_sched).mark_line().encode(
    x=alt.X('service_date', title=_dla_utils.labeling('service_date')),
    y=alt.Y('pct_ran', title=_dla_utils.labeling('pct_ran')),
    color='route_id',
    strokeDash='route_id',
)
line_chart = styleguide.preset_chart_config(line)
line_chart = _dla_utils.add_tooltip(line_chart, 'route_id', 'pct_ran')
line_chart.properties(width=600)

alt.Chart(...)

In [88]:
rt_sched>>filter(_.pct_ran<.5)>>count(_.route_id)

,route_id,n
0,3489,1
1,3491,1
2,3501,40
